In [1]:
import sys
sys.path.append("..")

from qiskit.providers.fake_provider import Fake27QPulseV1, Fake5QV1, Fake20QV1
from src.original_sabre.sabre import run_sabre
from src.isl_routing.mapping.routing import POLY_QMAP
from src.isl_routing.utils.isl_data_loader import *
from src.isl_routing.utils.circuit_utils import *
from time import time

In [2]:
edges = Fake27QPulseV1().configuration().coupling_map

In [3]:

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(instance="ibm-q/open/main")
 
backend = service.backend("ibm_sherbrooke")


backend_name = backend.name
edges = backend.configuration().coupling_map


In [4]:
file_path = "../benchmarks/polyhedral/queko-bss-53qbt/53QBT_600CYC_QSE_6.json"
data = json_file_to_isl(file_path)


In [5]:
run_sabre(data, edges)  

{'SabreLayout + SingleTrialSWAP': 1727,
 'SabreLayout + MultiTrialSWAP': 1363,
 'TrivialLayout + SingleTrialSWAP': 5253,
 'TrivialLayout + MultiTrialSWAP': 4913}

In [6]:
poly_mapper = POLY_QMAP(edges, data)

In [8]:
poly_mapper.run(heuristic_method="decay",initial_mapping_method="trivial",verbose=1)

Executing Gates: 100%|██████████| 6361/6361 [00:37<00:00, 171.25it/s]


6338

In [9]:
poly_mapper.run(heuristic_method="more_excuted",initial_mapping_method="trivial",verbose=1)

Executing Gates: 100%|██████████| 6361/6361 [00:29<00:00, 213.46it/s]


5101

In [ ]:
poly_mapper.run(heuristic_method="single-decay",initial_mapping_method="trivial",verbose=1)

In [14]:
#poly_mapper.run(heuristic_method="lookahead",initial_mapping_method="trivial")

159

In [15]:
poly_mapper.run(heuristic_method="max_focus",initial_mapping_method="trivial")

130

2572

In [114]:
poly_mapper.run()

120

In [118]:
results = {
    "multi_isl": [],
    "decay": [],
    "more_excuted": [],
    "single-decay": [],
    "max_focus": [],
    "sabre_single": [],
    "sabre_multi": []
}

In [123]:
from tqdm import tqdm
for i in range(800, 1000, 100):
    for j in tqdm(range(10), desc=f"Processing QSE {i}CYC", leave=False):
        file_path = f"../benchmarks/polyhedral/queko-bss-16qbt/16QBT_{i}CYC_QSE_{j}.json"
        data = json_file_to_isl(file_path)   
        poly_mapper = POLY_QMAP(edges, data)

        
        d = poly_mapper.run(heuristic_method="decay")
        results["decay"].append(d)
        m = poly_mapper.run(heuristic_method="more_excuted")
        results["more_excuted"].append(m)
        s = poly_mapper.run(heuristic_method="single-decay")
        results["single-decay"].append(s)
        f = poly_mapper.run(heuristic_method="max_focus")
        results["max_focus"].append(f)
        
        results["multi_isl"].append(min(d,m,s,f))

        sabre_single, sabre_multi = run_sabre(data, edges)
        results["sabre_single"].append(sabre_single)
        results["sabre_multi"].append(sabre_multi)


-------------------------

In [132]:
multi_better = 0
sabre_better = 0
equal = 0
total = len(results["multi_isl"])  # Total number of trials

for multi, sabre in zip(results["multi_isl"], results["sabre_multi"]):
    if multi < sabre:
        multi_better += 1
    elif sabre < multi:
        sabre_better += 1
    else:
        equal += 1

# Calculate percentages
multi_percentage = (multi_better / total) * 100 if total > 0 else 0
sabre_percentage = (sabre_better / total) * 100 if total > 0 else 0
equal_percentage = (equal / total) * 100 if total > 0 else 0

print(f"our solution better: {multi_better} times ({multi_percentage:.2f}%)")
print(f"sabre multi better: {sabre_better} times ({sabre_percentage:.2f}%)")
print(f"Equal performance: {equal} times ({equal_percentage:.2f}%)")


our solution better: 55 times (62.50%)
sabre multi better: 29 times (32.95%)
Equal performance: 4 times (4.55%)


In [128]:
import plotly.graph_objects as go



fig = go.Figure()

methods = list(results.keys())[::-1]

# Add each line with visibility toggle
for method, values in results.items():
    fig.add_trace(go.Scatter(
        x=list(range(40)), y=values,
        mode='lines+markers',
        name=method,
        visible=True  # Initially all are visible
    ))

# Add interactive buttons
fig.update_layout(
    updatemenus=[{
        "buttons": [
            {"label": "Show All", "method": "update", "args": [{"visible": [True] * len(results)}]},
            {"label": "Hide All", "method": "update", "args": [{"visible": [False] * len(results)}]}
        ] + [
            {"label": method, "method": "update",
             "args": [{"visible": [name == method for name in results]}]}
            for method in results
        ],
        "direction": "down",
        "showactive": True,
    }]
)

fig.update_layout(title="Performance of Heuristic Methods", xaxis_title="Trial Index", yaxis_title="Performance Metric")
fig.show()
